<a href="https://colab.research.google.com/github/hsuanchia/PTT-Gossiping-Chatbot/blob/main/gossip_chatbot_implement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 因為我使用CKIP，所以每次可能都需要重新下載一次，很快
! pip install ckiptagger[tagger]

In [ ]:
import pickle
import numpy as np
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from ckiptagger import WS

#Import

In [ ]:
model_path = '/content/drive/MyDrive/NLP/Gossip_chatbot_9.h5'
model = load_model(model_path)

OSError: ignored

In [ ]:
ws = WS("/content/drive/MyDrive/data") # CKIP的斷詞套件，如果沒有可能需要另外下載

Q_word_to_index = pickle.load(open(f'/content/drive/MyDrive/NLP/Q_word-index.pkl', 'rb'))
Q_index_to_word = pickle.load(open(f'/content/drive/MyDrive/NLP/Q_index-word.pkl', 'rb'))
A_word_to_index = pickle.load(open(f'/content/drive/MyDrive/NLP/A_word-index.pkl', 'rb'))
A_index_to_word = pickle.load(open(f'/content/drive/MyDrive/NLP/A_index-word.pkl', 'rb'))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:909: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


#Initialization parameters

In [ ]:
start = '<sos>'
end = '<end>' 
unk = '<unk>'
pad = '<pad>'
segment_delimiter = {'?', '？', '!', '！', ' ', '。', ',', ',','，', ';', ':', '、', '.', '(', ')', '「', '」', '（', '）'}

In [ ]:
Q_word_to_index[start] = 31652
Q_word_to_index[end] = 31653
Q_word_to_index[unk] = 31654
Q_index_to_word[31652] = start
Q_index_to_word[31653] = end
Q_index_to_word[31654] = unk

A_word_to_index[start] = 36397
A_word_to_index[end] = 36398
A_word_to_index[unk] = 36399
A_index_to_word[36397] = start
A_index_to_word[36398] = end
A_index_to_word[36399] = unk

Q_max_length = 30 #q_data max = 30, a_data max = 59
A_max_length = 60
Q_vocsize = len(Q_word_to_index)
A_vocsize = len(A_word_to_index)

# Data preprocessing

In [ ]:
def splitsentence(sentence):
  return ws(sentence_list=[sentence],sentence_segmentation = True, segment_delimiter_set=segment_delimiter)

In [ ]:
#將文字轉成integer sequence
def fit_sentence(sentence,word2index,max_length): 
  ans = []
  for i in sentence:
    if i in word2index.keys():
      ans.append(word2index[i])
    else:
      ans.append(word2index[unk])
  seq = pad_sequences([ans],maxlen=max_length,padding='post')
  return seq

# Chat Bot

In [ ]:
# 在開始跑chat bot之前要在上面import的地方import model，以及上面的所有code都要跑過
while True:
  question = input('公威阿: ')
  if question == '滾': # 輸入 '滾' 即可結束Chat Bot
    print('...88')
    break
  ans_seq = ''
  cur_token = start
  word_count = 1
  target_seq = np.zeros((1,A_max_length),dtype='int64')
  target_seq[0,0] = A_word_to_index[start]
  q = splitsentence(question)
  q = fit_sentence(q[0],Q_word_to_index,Q_max_length)
  while word_count < A_max_length:
    decoder_output = model.predict([q,target_seq])
    ind = np.argmax(decoder_output[0,word_count])
    cur_token = A_index_to_word[ind]
    if cur_token == end:
      break
    ans_seq += cur_token
    target_seq[0,word_count] = ind
    word_count += 1
  print(ans_seq)

公威阿: 有沒有疫苗的掛?
 你是不是不是嗎？
公威阿: 你484確診了?
 你是不是不是你的問題
公威阿: 資工肥宅是防疫小尖兵嗎?
 你是不是不是嗎？


# Test

In [ ]:
Q = '有沒有疫苗的掛?'
Q = splitsentence(Q)
print(Q)
Q = fit_sentence(Q[0],Q_word_to_index,Q_max_length)
A = fit_sentence([start],A_word_to_index,A_max_length)
print(Q,A)
pre = model.predict([Q,A])


[['有', '沒', '有', '疫苗', '的', '掛', '?']]
[[  6  14   6  20   2 174   4   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0]] [[36397     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0]]
 


In [ ]:
p

{1: ' ', 2: '.', 3: '的', 4: '是', 5: '，', 6: '不', 7: '你', 8: '就', 9: '了', 10: '有', 11: '都', 12: '我', 13: '一', 14: '?', 15: '沒', 16: '要', 17: '人', 18: '？', 19: '個', 20: '在', 21: '會', 22: '!', 23: '啊', 24: '還', 25: '也', 26: '嗎', 27: '說', 28: '這', 29: '好', 30: '看', 31: '很', 32: '跟', 33: '去', 34: '他', 35: '那', 36: '吧', 37: ',', 38: '被', 39: '打', 40: '台灣', 41: '想', 42: '才', 43: '可以', 44: '啦', 45: '到', 46: '天', 47: '現在', 48: '！', 49: '只', 50: '用', 51: '知道', 52: '過', 53: '次', 54: '。', 55: '買', 56: '能', 57: '上', 58: '什麼', 59: '又', 60: '自己', 61: '疫苗', 62: '吃', 63: '給', 64: '多', 65: '沒有', 66: '不會', 67: '大', 68: '來', 69: '真的', 70: '再', 71: '家', 72: '做', 73: '三', 74: '堆', 75: '年', 76: '種', 77: '但', 78: '誰', 79: '不要', 80: '死', 81: '政府', 82: '兩', 83: '缺', 84: '哈', 85: '幾', 86: '太', 87: '先', 88: '怎麼', 89: '哪', 90: '每', 91: '電', 92: '還是', 93: '喔', 94: '錢', 95: '這樣', 96: '對', 97: '叫', 98: '得', 99: '最', 100: '事', 101: '問', 102: '快', 103: '中國', 104: '覺得', 105: '黨', 106: '應該', 107: '、', 108: '小', 109: '講',

In [ ]:
for i in pre:
  for j in i:
    word = np.argmax(j)
    print(word)
    print(A_index_to_word[word])